In [1]:
### This is a module used to generate tabulated Gaussian integrations in 2D.
### Author: Yufei "Motherfucker" Pei

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import itertools
from PIL import Image
from scipy.stats import logistic


# Name of the hdf file that contain the data we need
f_name = 'sxro6416-r0504.h5'

# Open the hdf5 file, use the path to the images to extrate the data and place
# it in the image data object for further manipulation and inspection.
datafile = h5py.File(f_name, 'r')
image_data = []
for i in itertools.count(start=0):
    d = datafile.get(f'Configure:0000/Run:0000/CalibCycle:{i:04d}/Princeton::FrameV2/SxrEndstation.0:Princeton.0/data')
    if d is not None:
        # actual image is at first index
        image_data.append(d[0])
    else:
        break

#print(image_data[1])
# Tell me how many images were contained in the datafile
print(f"loaded {len(image_data)} images")


#for i in range(20):
    #plt.imshow(image_data[i])
    #plt.savefig('image'+str(i)+'.png',dpi=1000)
    #image = Image.fromarray(image_data[i]).convert("L")
    #image.save("image"+str(i)+".png")

# Plot a good dataset - here index 8 (but there are others too!)
#misc.imshow(image_data[0])
#misc.show()

#image = Image.fromarray(image_data[0]).convert("L")
#image.save("out.png")

# The histogram of the data will help show possible single photon hits
plt.hist(image_data[1].flatten(), bins=100)
plt.yscale('log')


loaded 20 images


In [3]:
dat=image_data[1]
dat.shape
test_dat=dat[242:262,1246:1288].astype(int)-50
test_dat_2=dat[492:525,1308:1340].astype(int)-50
test_height=test_dat_2.shape[0]
test_width=test_dat_2.shape[1]
print(test_height,test_width)

33 32


In [4]:
### Define the distribution function of single photon events.

#Here, such distribution is modeled as Gaussian. Later this will be justified/replaced with more justified forms based on
#literatures and/or physical simulations

def dist_f_gaussian(x,y,x0,y0,s1,s2,A,t,cutoff=2):
    x1=(x-x0)*np.cos(t)-(y-y0)*np.sin(t)
    y1=(x-x0)*np.sin(t)+(y-y0)*np.cos(t)
    #print(x1,y1,x0,y0,s1,s2,A,t,cutoff)
    if(x1>=cutoff*s1 or y1>=cutoff*s2):
        return 0
    else:
        return A/s1/s2*np.exp(-x1**2/(2*s1**2)-y1**2/(2*s2**2))

#With respect to this distribution, we create a class of single photon events(SPE):
from scipy.integrate import simps
import math

class gaussian_spe:
    'A distribution of intensity caused by a single photon event'
    
    intensity_matrix=np.zeros((test_height,test_width))
    
    dist_fn='gaussian'
    
    def __init__(self, y0, x0, s1, s2, A, t=0, cutoff=2):
        self.x0 = x0
        self.y0 = y0
        self.s1 = s1
        self.s2 = s2
        self.A = A
        self.t = t
        self.cutoff = cutoff
        self.intensity_matrix=np.zeros((test_height,test_width))
        s=max([s1,s2])+0.5
        for i in range(test_height):
            for j in range(test_width):
                if(not((i<x0+cutoff*s and i>x0-cutoff*s) and (j<y0+cutoff*s and j>y0-cutoff*s))):
                    continue
                """
                #The simpson rule method
                
                x=np.linspace(i,i+1,sample_length)
                y=np.linspace(j,j+1,sample_length)
                #zz = self.dist(x.reshape(-1,1),y.reshape(1,-1))
                zz=np.zeros((sample_length,sample_length))
                for xi in range(sample_length):
                    for yi in range(sample_length):
                        zz[xi,yi]=self.dist(x[xi],y[yi])
                dist=lambda x,y: self.dist(x,y)
                #self.intensity_matrix[i,j]=integrate.dblquad(dist,i,i+1,lambda x:j,lambda x:j+1,epsabs=1e-4, epsrel=1e-4)[0]
                self.intensity_matrix[i,j]=simps([simps(zz_x,x) for zz_x in zz],y) 
                """
                
                #The MUCH SIMPLER erf method
                if(math.sqrt((i+0.5-self.x0)**2+(j+0.5-self.y0)**2)>cutoff*2*s):
                    self.intensity_matrix[i,j]=0
                else:
                    self.intensity_matrix[i,j]=self.A*self.s1**2*self.s2**s2*(math.erf(-self.x0+i/math.sqrt(2))-math.erf(-self.x0+i+1/math.sqrt(2)))*math.erf(-self.y0+j/math.sqrt(2))-math.erf(-self.y0+j+1/math.sqrt(2))
                
    
    def dist(self,x,y):
        return dist_f_gaussian(x,y,self.x0,self.y0,self.s1,self.s2,self.A,self.t,self.cutoff)
    
    def new_value(self, y0, x0, s1, s2, A, t=0, cutoff=2):
        self.x0 = x0
        self.y0 = y0
        self.s1 = s1
        self.s2 = s2
        self.A = A
        self.t = t
        self.cutoff = cutoff
        s=max([s1,s2])+1.5
        for i in range(test_height):
            for j in range(test_width):
                if(not((i<x0+cutoff*s and i>x0-cutoff*s) and (j<y0+cutoff*s and j>y0-cutoff*s))):
                    continue
                x=np.linspace(i,i+1,sample_length)
                y=np.linspace(j,j+1,sample_length)
                #zz = self.dist(x.reshape(-1,1),y.reshape(1,-1))
                zz=np.zeros((sample_length,sample_length))
                for xi in range(sample_length):
                    for yi in range(sample_length):
                        zz[xi,yi]=self.dist(x[xi],y[yi])
                dist=lambda x,y: self.dist(x,y)
                #self.intensity_matrix[i,j]=integrate.dblquad(dist,i,i+1,lambda x:j,lambda x:j+1,epsabs=1e-4, epsrel=1e-4)[0]
                self.intensity_matrix[i,j]=simps([simps(zz_x,x) for zz_x in zz],y) 
    
    def copy(self):
        return self
                
from scipy import integrate
import time

# The error function. It is defined as ...; the minimisation of which is the ultimate goal of this part of the code. 
def error(spe__array,dat_tag,input_dat=test_dat):
    matrix=input_dat[(dat_tag.param()[0]):(dat_tag.param()[2]+1),(dat_tag.param()[1]):(dat_tag.param()[3]+1)]
    #print(matrix)
    height=matrix.shape[0]
    width=matrix.shape[1]
    tot_intensity_matrix=np.zeros((height,width))
    err=0
    for i in range(height):
        for j in range(width):
            for event in spe__array:
                tot_intensity_matrix[i,j]=tot_intensity_matrix[i,j]+event.intensity_matrix[dat_tag.param()[0]+i,dat_tag.param()[1]+j]
            err=err+(matrix[i,j]-tot_intensity_matrix[i,j])**2
        
    return err